In [ ]:
!pip install -q split-folders

In [ ]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import os
import splitfolders
import copy

In [ ]:

input_folder = '/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset' 

split_output_folder = '/kaggle/working/dataset_split'

print(f"Membagi dataset dari '{input_folder}'...")

if os.path.exists(split_output_folder):
    os.system(f'rm -rf {split_output_folder}')
    
splitfolders.ratio(input_folder, output=split_output_folder, seed=42, ratio=(.8, .2))
print("Pembagian dataset selesai.")

In [ ]:
# --- Konfigurasi Training ---
DATA_DIR = '/kaggle/working/dataset_split'
MODEL_SAVE_PATH = '/kaggle/working/cnn_tea_disease_classifier.pth'
CLASS_NAMES_SAVE_PATH = '/kaggle/working/class_names.txt'
NUM_EPOCHS = 25
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0.01
IMAGE_SIZE = 224

# --- Transformasi Data ---
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- Membuat DataLoader ---
train_dataset = datasets.ImageFolder(root=os.path.join(DATA_DIR, 'train'), transform=train_transform)
val_dataset = datasets.ImageFolder(root=os.path.join(DATA_DIR, 'val'), transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

class_names = train_dataset.classes
num_classes = len(class_names)

print("DataLoader dan transformasi siap.")
print(f"Jumlah kelas terdeteksi: {num_classes} -> {class_names}")

In [ ]:
# --- Inisialisasi Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {device}")

# --- Inisialisasi Model (ResNet50) ---
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(num_ftrs, num_classes)
)
model = model.to(device)

# --- Inisialisasi Komponen Training ---
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

print("Model, optimizer, dan loss function siap.")

In [ ]:
# --- Training Loop ---
print("\n--- MEMULAI TRAINING MODEL ---")
best_accuracy = 0.0

for epoch in range(NUM_EPOCHS):
    model.train()
    total_train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        total_train_loss += loss.item()
    
    avg_train_loss = total_train_loss / len(train_loader)
    
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {avg_train_loss:.4f}, Val Accuracy: {accuracy:.2f}%")
    
    # Simpan model hanya jika akurasinya lebih baik
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"    -> Akurasi meningkat, model disimpan.")

    scheduler.step()

print(f"\n--- Training Selesai. Akurasi terbaik yang disimpan: {best_accuracy:.2f}% ---")

In [ ]:
with open(CLASS_NAMES_SAVE_PATH, 'w') as f:
    for item in class_names:
        f.write("%s\n" % item)

print(f"Model disimpan di: {MODEL_SAVE_PATH}")
print(f"Nama kelas disimpan di: {CLASS_NAMES_SAVE_PATH}")